matching between transcripts and audios

In [7]:
import pandas as pd
import numpy as py
from multiprocessing import Process, JoinableQueue, Manager

In [8]:
transcripts = pd.read_csv("transcript_metadata.csv", encoding = "ISO-8859-1")
audios = pd.read_csv("audio_metadata.csv")

In [4]:
transcripts["TextDateTime"] = transcripts["TextDateTime"].astype(str)
transcripts["Year"] = transcripts["Year"].astype(str)
# transcripts["Quarter"] = pd.to_datetime(transcripts["TextDateTime"]).dt.quarter
# transcripts["Quarter"] = transcripts["Quarter"].fillna(0)
# transcripts["Quarter"] = transcripts["Quarter"].astype(int)

fixing the date format in audios

In [5]:
dates_list = []
audios["Date"] = audios["Date"].astype(str)
for date in audios["Date"]:
    date_list = date.split("/")
    if len(date_list[0]) == 1:
        date_list[0] = "0" + date_list[0]
    if len(date_list[1]) == 1:
        date_list[1] = "0" + date_list[1]
    dates_list.append("/".join(date_list))
audios["Date"] = pd.DataFrame(dates_list)

add a Date field to transcripts in MM/DD/YY

In [6]:
date_list = []
for date in transcripts.TextDateTime:
    try:
        temp = date[5:7]+ "/" + date[8:10] + "/" + date[0:4]
    except BaseException:
        date_list.append(0)
    else:
        date_list.append(temp)
transcripts["Date"] = pd.DataFrame(date_list)

In [16]:
transcripts.head()

,Name,Exchange,Symbol,Description,Month,Day,Year,Time,APM,TimeZone,TextDateTime,Person,Title/Institution,Role,Date
0,"First Solar, Inc.",NASDAQ,FSLR,"First Solar, Inc. (FSLR) CEO Mark Widmar on 20...",December,11,2018,4:30,PM,ET,2018-12-11 16:30 ET,Steve Haymore,Investor Relations,E,12/11/2018
1,"First Solar, Inc.",NASDAQ,FSLR,"First Solar, Inc. (FSLR) CEO Mark Widmar on 20...",December,11,2018,4:30,PM,ET,2018-12-11 16:30 ET,Mark Widmar,Chief Executive Officer,E,12/11/2018
2,"First Solar, Inc.",NASDAQ,FSLR,"First Solar, Inc. (FSLR) CEO Mark Widmar on 20...",December,11,2018,4:30,PM,ET,2018-12-11 16:30 ET,Alex Bradley,Chief Financial Officer,E,12/11/2018
3,"First Solar, Inc.",NASDAQ,FSLR,"First Solar, Inc. (FSLR) CEO Mark Widmar on 20...",December,11,2018,4:30,PM,ET,2018-12-11 16:30 ET,Philip Shen,ROTH Capital Partners,A,12/11/2018
4,"First Solar, Inc.",NASDAQ,FSLR,"First Solar, Inc. (FSLR) CEO Mark Widmar on 20...",December,11,2018,4:30,PM,ET,2018-12-11 16:30 ET,Brian Lee,Goldman Sachs,A,12/11/2018


In [17]:
audios.head()

,Ticker,Quarter,Year,Name,Act,Est,Date,Time,Timezone,Length,Filename
0,OHRP,Q4,2017,"Ohr Pharmaceuticals, Inc.",Act -0.08,Est -0.08,12/15/2017,8:30 am,EST,9:55,"Q4_2017_Ohr_Pharmaceuticals,_Inc._Earnings_Con..."
1,BOOT,Q3,2018,"Boot Barn Holdings, Inc.",NaN,NaN,01/31/2018,4:30 pm,EST,0:10,"Q3_2018_Boot_Barn_Holdings,_Inc._Earnings_Conf..."
2,PAYX,Q2,2018,"Paychex, Inc.",Act 0.59,Est 0.594,12/21/2017,9:30 am,EST,1:20:13,"Q2_2018_Paychex,_Inc._Earnings_Conference_call..."
3,AU,Q4,2017,Anglo Gold Ashanti Limited,NaN,NaN,02/21/2018,8:00 am,EST,0:10,Q4_2017_Anglo_Gold_Ashanti_Limited_Earnings_Co...
4,CYOU,Q4,2017,Changyou.Com Limited,NaN,NaN,02/21/2018,7:00 am,EST,0:10,Q4_2017_Changyou.Com_Limited_Earnings_Conferen...


KeyError: 10

In [9]:
manager1 = Manager()
a = [None] * len(transcripts)
filename_list = manager1.list(a)

manager2 = Manager()
b = [0] * len(transcripts)
found_number = manager2.list(a)

q = JoinableQueue()

for transcript in transcripts.itertuples(index=True, name='Pandas'):
    q.put(transcript)

def match(q):
    while not q.empty():
        transcript = q.get()
        flag = False
        for audio in audios.loc[(audios["Ticker"] == transcript[3]) & (audios["Date"] == transcript[15])].itertuples(index=True, name='Pandas'):
            flag = True
            filename_list[transcript[0]] = audio[11]
            found_number[transcript[0]] += 1
        if not flag:
            filename_list[transcript[0]] = "NaN"
        if transcript[0] % 10000 == 0:
            print(transcript[0])
        q.task_done()

num_threads = 8
for i in range(num_threads):
    print('Starting thread ', i)
    worker = Process(target=match, args=(q,))
    worker.start()
q.join()
print('All tasks completed.')

transcripts["the corresponding audio file name"] = pd.DataFrame(filename_list)
transcripts["# of match"] = pd.DataFrame(found_number)

KeyboardInterrupt: 

export

In [ ]:
transcripts.to_csv("final_multithread.csv", index=False)

audio conversion

install ffmpeg, pydub before using

m4a -> mp3 increase file size a HELL LOT, be aware when using

In [1]:
from pydub import AudioSegment
import os
from multiprocessing import Process, JoinableQueue, cpu_count

config the input output directories manually

In [2]:
path = os.getcwd()
audio_input_folder = "sample"
audio_output_folder = "audio_output"

single file conversion

In [ ]:
sound = AudioSegment.from_file(path + "/" + audio_input_folder + "/" + "1Q_2013_Pike_Electric_Corporation_Earnings_Conference_Call.m4a")
sound.export(path + "/" + audio_output_folder + "/" + "test.mp3", format="mp3")

batch conversion

BE AWARE OF THE LAGGGGGGGG

In [3]:
filelist = os.listdir(path + "/" + audio_input_folder)

file checking

In [4]:
for file in filelist:
    if len(file) <= 5:
        filelist.remove(file)
        print(file + " is wrong")
    elif file[-4:] != ".m4a":
        filelist.remove(file)
        print(file + " is wrong")

.DS_Store is wrong


In [6]:
q = JoinableQueue()
for file in filelist:
    q.put(file)

def convert(q, i):
    while not q.empty():
        file = q.get()
        print("Thread " + str(i) + " now processing " + file)
        sound = AudioSegment.from_file(path + "/" + audio_input_folder + "/" + file)
        mp3name = file[:-4] + ".wav"
        sound.export(path + "/" + audio_output_folder + "/" + mp3name, format="wav")
        q.task_done()

num_threads = cpu_count()
for i in range(num_threads):
    print('Starting thread ', i)
    worker = Process(target=convert, args=(q, i))
    worker.start()
q.join()
print('All tasks completed.')

Starting thread  0
Starting thread  1
Thread 0 now processing 3D_Systems_Corp_Analyst_and_Investor_Meeting.m4a
Starting thread  2
Thread 1 now processing 2012_Oshkosh_Corporation_Analyst_Day.m4a
Starting thread  3
Thread 2 now processing 2012_A&B_Real_Estate_Sell-Side_Analyst_Day.m4a
Starting thread  4
Thread 3 now processing 2012_Pinnacle_West_Analyst_Day.m4a
Starting thread  5
Thread 4 now processing 3_M_Company_At_Sanford_C._Bernstein_&_Co._Strategic_Decisions_Conference.m4a
Starting thread  6
Thread 5 now processing 1Q12_Earnings_Conference_Call_(Replay).m4a
Starting thread  7
Thread 7 now processing 4Q_and_Full_Year_2010_Goodrich_Corporation_Earnings_Conference_Call.m4a
Thread 6 now processing 1Q_2013_Pike_Electric_Corporation_Earnings_Conference_Call.m4a
Thread 6 now processing 2012_ASCO_Investor_Event.m4a
Thread 5 now processing 4Q11_Earnings_Conference_Call.m4a
Thread 4 now processing 4Q_FY11_Cardinal_Health_Inc_Earnings_Conference_Call.m4a
Thread 7 now processing 2011_Assurant